In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from indexing.processor import IndexingProcessor
from search.qdrant import QdrantSearchEngine
from search.bm25 import ElasticsearchBM25
import json
from search.hybrid_search import search

In [ ]:

qdrant_engine = QdrantSearchEngine()
es_engine =  ElasticsearchBM25()
await es_engine.initialize()


processor = IndexingProcessor(qdrant_engine=qdrant_engine, es_engine=es_engine)

In [ ]:
with open('data/meeting.json', 'r') as f:
    data = json.load(f)

In [ ]:
df =pd.DataFrame(data['segments'])

In [ ]:
speakers = df['speaker'].dropna().unique().tolist()
start_datetime = pd.to_datetime(df['absolute_start_time'].min())
content_id = '1'

In [ ]:
r = await processor._merge_chunks(df, content_id, start_datetime, speakers)
await processor._index_to_search_engines(r[0], r[1])

In [ ]:
await search('meeting notetaker',qdrant_engine=qdrant_engine, es_engine=es_engine, k=1)
